### 기본 세팅

In [ ]:
# GPU 성능을 높이기 위한 라이브러리
!pip install -U accelerate==0.29.3 peft==0.10.0 bitsandbytes==0.45.2 # 처음 버전: 0.43.1
# !pip install --upgrade transformers bitsandbytes accelerate
# !pip install peft==0.10.0
!pip install fsspec==2024.12.0 gcsfs==2024.12.0 # 처음버전: fsspec==2024.3.1
!pip install --upgrade datasets
!pip install -U transformers==4.41.0 trl==0.8.6

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 76.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM, AutoTokenizer, # 사전 학습된 모델과 토크나이저를 불러오기 위한 클래스
    TrainingArguments, # 트레이닝 과정의 다양한 파라미터 세팅
    BitsAndBytesConfig, # 모델의 메모리 최적화 관련 설정
    pipeline,
    logging
)
from peft import LoraConfig
from trl import SFTTrainer

- 한국어 데이터셋을 훈련받은 LLama 모델 사용

In [ ]:
base_model_path = "beomi/Llama-3-Open-Ko-8B"

In [ ]:
base_model_path

'beomi/Llama-3-Open-Ko-8B'

In [ ]:
# GPU device_capablity가 8 이상이면 고급기능 사용.
if torch.cuda.get_device_capability()[0] >= 8:
    !pip install -qqq flash-attn
    attn_implementation = "flash_attention_2"
    torch_dtype = torch.bfloat16
else:
    attn_implementation = "eager"
    torch_dtype = torch.float16

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 50.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


- QLoRA(양자화된 모델 로드하도록 설정)

In [ ]:
# 메모리와 계산 속도 최적화
# 4비트 양자화된 모델
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

- 모델 로드

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    base_model_path,
    quantization_config=quant_config,
    # device_map="auto"
    device_map={"":0}
)
model.config.use_cache = False
model.config.pretraining_tp = 1

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/698 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00006.safetensors:   0%|          | 0.00/3.00G [00:00<?, ?B/s]

model-00002-of-00006.safetensors:   0%|          | 0.00/2.94G [00:00<?, ?B/s]

...

model-00004-of-00006.safetensors:   0%|          | 0.00/2.94G [00:00<?, ?B/s]

model-00005-of-00006.safetensors:   0%|          | 0.00/2.94G [00:00<?, ?B/s]

model-00006-of-00006.safetensors:   0%|          | 0.00/1.29G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

- 토크나이저 설정

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    base_model_path,
    trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # 문장 벡터화할 때 길이 맞춤

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


- PEFT (LoRA)

In [ ]:
peft_params = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

- 학습 파라미터 설정

In [ ]:
training_params = TrainingArguments(
    output_dir="/content/drive/MyDrive/healthcare project/results",
    num_train_epochs=20, # 총 학습 epoch 수(데이터셋을 몇 번 반복할지)
    per_device_train_batch_size=4, # 배치 크기(한번에 처리하는 데이터 개수)
    # 배치 크기만큼의 데이터를 4번 처리한 후 가중치 업데이트.
    # 배치 크기를 키우지 않아 메모리는 절약하지만 학습효과는 [batch_size= 설정 배치크기 * step] 처럼 동작
    gradient_accumulation_steps=1, # epochs랑 동시에 사용 못하므로 epochs를 1이상으로 했으면 이건 1로
    gradient_checkpointing=True, # 메모리 절약용
    optim="paged_adamw_32bit", # 옵티마이저 설정
    save_steps=500,
    save_total_limit=1,  # 최근 1개 체크포인트만 유지(최근 1개만 저장)
    logging_steps=100,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False, # 원래 False 였음
    bf16=False,
    max_grad_norm=0.3, # 0.3 이상 변화 방지, 폭발적 그라디언트 방지
    max_steps=-1, # -1: 데이터셋 끝날 때까지 학습. 5000: 스텝 후 학습 종료
    warmup_ratio=0.03, # 전체 학습의 처음 3%동안 학습률 점진적으로 증가. 학습 초반에 서서히 학습
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

### 데이터 불러오기

In [ ]:
# # 허깅페이스 데이터
# # llama 데이터 로드
# dataset = load_dataset(dataset_dw, split="train")

# # 데이터 확인
# print(len(dataset), dataset[5])

In [ ]:
# 데이터셋 로드
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/healthcare project/headache_qa_matching.csv', index_col=0)
df

,disease_category,disease_name,intention,question,final_answer
0,응급질환,두통,예방,두통 예방을 위해 어떤 자세나 활동이 도움이 될까요?,만약 일상적인 예방 및 조절 조치를 취하는 것이 어렵거나 효과가 없다면 다음과 같은...
1,응급질환,두통,예방,스트레스를 많이 받는 사람이 두통을 예방하기 위해 어떤 생활습관을 가져야 할까요?,두통은 많은 사람들이 일상적으로 겪는 현상입니다. 일반적인 예방 방법으로는 규칙적인...
2,응급질환,두통,예방,대상포진 예방접종 후에 어지럼증과 두통이 예방접종과 관련이 있다면 어떻게 되나요?,만약 두통이 오랫동안 지속된다면 의사의 진단을 받아보는 것이 좋습니다. 스트레스와 ...
3,응급질환,두통,예방,갈색세포증과 관련하여 일상생활에서 신경써야할 점은 무엇인가요?,만약 두통이 오랫동안 지속된다면 의사의 진단을 받아보는 것이 좋습니다. 스트레스와 ...
4,응급질환,두통,예방,두통을 예방하기 위한 수면 자세에는 어떤 것들이 있을까요?,긴장성 두통은 일상 생활에서 자세나 습관에 따라 발생할 수 있습니다. 특히 수면 자...
...,...,...,...,...,...
3423,응급질환,두통,치료,두통과 스트레스 반응에 대한 급성 과민성을 진단하고 치료하는 방법에는 어떤 것들이 ...,두통은 일상 생활에서 자주 경험할 수 있는 증상입니다. 두통은 다양한 원인에 의해 ...
3424,응급질환,두통,치료,약국에서 구입 가능한 두통 치료제 중에서 어떤 것이 가장 효과적인지 알려주세요.,두통은 흔한 증상이며 많은 치료법이 있습니다. 두통은 치료하기 위해 여러 가지 방법...
3425,응급질환,두통,치료,만성 두통을 치료하기 위해 어떤 방법이 있는지 알려주세요.,두통은 흔한 증상이며 많은 치료법이 있습니다. 두통은 치료하기 위해 여러 가지 방법...
3426,응급질환,두통,치료,두통의 치료를 위해 어떤 약물이 사용될 수 있을까요?,두통의 치료에는 다양한 방법이 있습니다. 주로 약물 요법이나 식이요법을 통해 증상을...


In [ ]:
dataset = []
for idx, row in df.iterrows():
    # print(idx, q['question'])
    dataset.append({
        'text': f"<s>[INST] {row['question']} [/INST]{row['final_answer']}</s>"
    })

In [ ]:
dataset[0]

{'text': '<s>[INST] 두통 예방을 위해 어떤 자세나 활동이 도움이 될까요? [/INST]만약 일상적인 예방 및 조절 조치를 취하는 것이 어렵거나 효과가 없다면 다음과 같은 조언을 고려해 보세요. 먼저, 의사나 전문가와의 상담을 통해 개개인의 건강 상태와 일상 생활 방식에 대해 알아보세요. 두통을 예방하기 위해 취할 수 있는 조치들을 찾아보세요. 또한 스트레스 관리 기술을 배우고 실천하세요. 스트레스를 잘 다루는 방법을 익히고 적용하는 것이 중요합니다. 규칙적인 운동을 통해 신체의 기능과 유연성을 향상시키세요. 마지막으로, 충분한 수면을 취하는 것이 매우 중요합니다. 적절한 수면 패턴을 유지하고 신체의 활력을 회복하세요. 의사나 전문가의 조언을 들어 건강한 생활 습관을 만들어보세요. 생활 방식을 개선하고 스트레스를 완화하는 기술을 배우는 것이 두통 예방에 도움이 됩니다.</s>'}

- 데이터셋 형식으로 변환하기

In [ ]:
from datasets import Dataset

dataset = Dataset.from_list(dataset)

In [ ]:
len(dataset), dataset[0]

(3428,
 {'text': '<s>[INST] 두통 예방을 위해 어떤 자세나 활동이 도움이 될까요? [/INST]만약 일상적인 예방 및 조절 조치를 취하는 것이 어렵거나 효과가 없다면 다음과 같은 조언을 고려해 보세요. 먼저, 의사나 전문가와의 상담을 통해 개개인의 건강 상태와 일상 생활 방식에 대해 알아보세요. 두통을 예방하기 위해 취할 수 있는 조치들을 찾아보세요. 또한 스트레스 관리 기술을 배우고 실천하세요. 스트레스를 잘 다루는 방법을 익히고 적용하는 것이 중요합니다. 규칙적인 운동을 통해 신체의 기능과 유연성을 향상시키세요. 마지막으로, 충분한 수면을 취하는 것이 매우 중요합니다. 적절한 수면 패턴을 유지하고 신체의 활력을 회복하세요. 의사나 전문가의 조언을 들어 건강한 생활 습관을 만들어보세요. 생활 방식을 개선하고 스트레스를 완화하는 기술을 배우는 것이 두통 예방에 도움이 됩니다.</s>'})

In [ ]:
# SFTTrainer 클래스를 사용해 파인튜닝
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_params,
    dataset_text_field="text",
    max_seq_length=None, # 입력 시퀀스의 최대 길이를 별도로 제한하지 않음.
    tokenizer=tokenizer,
    args=training_params,
    packing=False,
)

/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:246: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/3428 [00:00<?, ? examples/s]

In [ ]:
# 학습시기키
trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
100,1.236200
200,1.068200
300,0.996500
400,0.899600
500,0.828900
600,0.793900
700,0.712900
800,0.675700
900,0.614200
1000,0.587300


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `

Step,Training Loss
100,1.236200
200,1.068200
300,0.996500
400,0.899600
500,0.828900
600,0.793900
700,0.712900
800,0.675700
900,0.614200
1000,0.587300


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `

TrainOutput(global_step=17140, training_loss=0.20080266126812965, metrics={'train_runtime': 46082.8624, 'train_samples_per_second': 1.488, 'train_steps_per_second': 0.372, 'total_flos': 7.005930569134572e+17, 'train_loss': 0.20080266126812965, 'epoch': 20.0})

- 질문하기

In [ ]:
query = '두통의 증상이 심해질 경우 어떤 치료 방법이 효과적일까요?'
pipe = pipeline(task='text-generation',
                model=model,
                tokenizer=tokenizer,
                max_length = 256)
result = pipe(f"<s>[INST] {query} [/INST]")
answer_result = result[0]['generated_text']

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


In [ ]:
answer_result

'<s>[INST] 두통의 증상이 심해질 경우 어떤 치료 방법이 효과적일까요? [/INST]두통은 다양한 이유로 발생할 수 있습니다. 두통은 일상적인 활동이나 스트레스와 관련된 요인으로 인해 증상이 나타나는 경향이 있습니다. 예를 들어, 두통은 피로, 우울감, 불안과 같은 감정 변화와 연관될 수 있습니다. 두통은 매우 흔한 증상이며, 다양한 증상과 함께 나타날 수 있습니다. 만약 두통이 지속된다면 의료 전문가와 상담하여 원인을 파악하고 적절한 치료 방법을 찾는 것이 중요합니다.</two>'

In [ ]:
query = '두통이라는 질병에서 머리 통증 이외에 어떤 증상들이 더 일반적으로 나타나나요?'
pipe = pipeline(task='text-generation',
                model=model,
                tokenizer=tokenizer,
                max_length = 256)
result = pipe(f"<s>[INST] {query} [/INST]")
answer_result = result[0]['generated_text']
answer_result

'<s>[INST] 두통이라는 질병에서 머리 통증 이외에 어떤 증상들이 더 일반적으로 나타나나요? [/INST]두통은 사람들이 자주 겪는 두통의 일반적인 형태입니다. 두통은 머리의 한쪽 부분에서 발생하는 통증을 의미합니다. 이 통증은 다양한 부위에서 나타날 수 있으며, 근육통, 관절통, 피로감, 불안감, 흥분 등의 다른 증상과 함께 나타날 수 있습니다. 일부 경우, 두통은 얼굴, 목, 팔, 어깨, 다리 등의 다양한 부위에서 나타날 수 있습니다. 일반적으로는 머리 통증이 가장 흔한 증상이지만, 때로는 다른 신체 부위에서도 두통을 느낄 수 있습니다. 두통은 매우 흔한 증상으로, 주로 머리의 한쪽 부분에서 발생합니다. 만약 두통이 심하거나 지속된다면, 의사와 상담하여 원인을 확인하고 적절한 치료를 받는 것이 좋습니다.</two>'

In [ ]:
query = '한쪽 머리가 아픈 증상은 어떤 증상이야?'
pipe = pipeline(task='text-generation',
                model=model,
                tokenizer=tokenizer,
                max_length = 256)
result = pipe(f"<s>[INST] {query} [/INST]")
answer_result = result[0]['generated_text']
answer_result

'<s>[INST] 한쪽 머리가 아픈 증상은 어떤 증상이야? [/INST]두통의 원인은 다양한데, 일반적으로 머리가 심하게 아픈 증상이 나타납니다. 두통은 뇌의 부위에 따라 다양한 원인으로 발생할 수 있습니다. 뇌막염으로 인한 두통은 두통이 발열, 오한, 구토와 함께 발생할 수 있고, 뇌수막염의 증상으로 나타날 수도 있습니다. 또한 뇌염은 고열, 오심, 구토와 함께 두통이 발생할 수 있고, 뇌수막염의 가능성도 있습니다. 증상에 따라 뇌종양, 뇌출혈, 뇌졸중, 뇌감염 등과 같은 질환도 두통을 유발할 수 있습니다. 그러나 일반적으로는 증상이 있는 경우에는 전문가의 조언을 받아 두통을 완화할 수 있도록 조치를 취해야 합니다.</br>두통이 지속되거나 심해진다면 의료전문가의 상담을 받아보시기 바랍니다.</br>두통이 심해지거나 지속될 경우 의료 전문가와 상담'

In [ ]:
query = '두통의 종류를 알려줘'
pipe = pipeline(task='text-generation',
                model=model,
                tokenizer=tokenizer,
                max_length = 256)
result = pipe(f"<s>[INST] {query} [/INST]")
answer_result = result[0]['generated_text']
answer_result

'<s>[INST] 두통의 종류를 알려줘 [/INST]두통은 일상 생활에서 자주 발생하는 흔한 증상 중 하나입니다. 하지만 증상은 원인과 함께 다양하게 나타날 수 있습니다. 일차성 두통의 원인 중 가장 흔한 것은 스트레스입니다. 긴장성 두통은 스트레스와 같은 외부적 요인으로 인해 발생하며, 한쪽 머리에서 시작해 서서히 다른 부위로 확산될 수 있습니다. 편두통은 특정한 상황이나 스트레스가 주 원인입니다. 일반적으로 편두통은 스트레스에 노출되거나 정서적인 변화가 발생할 때 악화될 수 있습니다. 두통은 원인에 따라 다양한 형태로 나타날 수 있습니다. 스트레스성 두통이나 긴장성 두통은 원인에 따라 적절한 치료와 예방법을 찾을 수 있습니다. 그러나 편두통은 치료가 어렵고 증상이 반복적으로 나타날 수 있으므로 조기에 치료하는 것이 중요합니다. 두통을 관리하기 위해서는 의사와 상담하여 원인을 파악하고 적절한 치료를 받는 것이 중요합니다'

- 학습된 데이터 저장하기

In [ ]:
trained_directory = "/content/drive/MyDrive/saved_trained_model"

# LoRA(PEFT) 모델 저장
trainer.model.save_pretrained(trained_directory)
tokenizer.save_pretrained(trained_directory)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


('/content/drive/MyDrive/saved_trained_model/tokenizer_config.json',
 '/content/drive/MyDrive/saved_trained_model/special_tokens_map.json',
 '/content/drive/MyDrive/saved_trained_model/tokenizer.json')

- 저장한 가중치와 기본모델을 불러와 모델 다시 생성(가중치와 기본 모델을 병합해서 저장하지 않음)

In [ ]:
# 기본 모델 저장
# PEFT(LoRA) 적용된 모델이라면 기본 모델의 가중치는 저장되지 않으므로 기본 모델도 저장해야함.
# LoRA는 기본 모델의 가중치를 직접 변경하지 않고, 추가적인 적은 수의 가중치만 학습하는 방식임.
# 그래서 save_pretrained()를 호출하면 LoRA의 추가 가중치만 저장됨.
# trainer.model.base_model.save_pretrained(f"{save_directory}/base_model")

In [ ]:
import os
import torch
from transformers import (
    AutoModelForCausalLM, AutoTokenizer, # 사전 학습된 모델과 토크나이저를 불러오기 위한 클래스
    TrainingArguments, # 트레이닝 과정의 다양한 파라미터 세팅
    BitsAndBytesConfig, # 모델의 메모리 최적화 관련 설정
    pipeline,
    logging
)
from peft import LoraConfig
from trl import SFTTrainer

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# 학습때 설정한 양자화 옵션을 불러올 때도 그대로 넣어줘야함.
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

# 기본 모델 로드
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_path,
    device_map={"":0},
    quantization_config = quant_config,  # 양자화 적용
    torch_dtype=torch.float16  # 양자화된 모델을 불러올 때 dtype 설정
    )

# LoRA 가중치 적용
# 불러온 기본 모델(base_model)에 LoRA 가중치를 적용
merged_model = PeftModel.from_pretrained(base_model, trained_directory)

# 토크나이저 불러오기
merged_tokenizer = AutoTokenizer.from_pretrained(trained_directory)

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


- 기본 모델과 가중치를 불러와 병합해서 저장

In [ ]:
# !pip install --upgrade transformers bitsandbytes accelerate

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

save_directory = "/content/drive/MyDrive/full_trained_model"

# LoRA 병합 (이제 LoRA 없이도 사용할 수 있음)
merged_model_for_save = merged_model.merge_and_unload()

# 전체 모델(기본 모델 + 학습한 가중치) 저장
merged_model_for_save.save_pretrained(save_directory)
merged_tokenizer.save_pretrained(save_directory)

/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/bnb.py:325: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


('/content/drive/MyDrive/full_trained_model/tokenizer_config.json',
 '/content/drive/MyDrive/full_trained_model/special_tokens_map.json',
 '/content/drive/MyDrive/full_trained_model/tokenizer.json')

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# 저장했던 모델 다시 로드
model_load = AutoModelForCausalLM.from_pretrained("/content/drive/MyDrive/full_trained_model", device_map="auto")
tokenizer_load = AutoTokenizer.from_pretrained("/content/drive/MyDrive/full_trained_model")

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# question = '<s>[INST]두통의 증상이 심해질 경우 어떤 치료 방법이 효과적일까요?[/INST]'

# # 토크나이징 (입력을 토큰으로 변환)
# # tokenizer가 반환하는 값은 dic 형태임. to(device)를 사용해 텐서들을 GPU나 CPU로 이동시킴
# inputs = tokenizer(question, return_tensors="pt").to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

# # 답변 생성
# with torch.no_grad():
#     output = model.generate(**inputs,
#                             max_length=256,
#                             do_sample=True,   # 샘플링 기반 생성
#                             top_p=0.95,       # 상위 95% 확률 분포에서 샘플링
#                             temperature=0.7   # 생성 다양성 조절 (낮을수록 보수적, 높을수록 창의적)
#                             )

# # 출력(디코딩). 토큰을 문자열로 변환
# response = tokenizer.decode(output[0], skip_special_tokens=True)

# # 결과 출력
# print("🤖 모델의 답변:", response)

🤖 모델의 답변: <s>[INST]두통의 증상이 심해질 경우 어떤 치료 방법이 효과적일까요?[/INST]두통은 뇌의 신경계가 예민해지면서 발생하는 질환입니다. 두통은 머리가 아픈 증상만 보이지 않으면서도, 증상이 심해질 경우에는 치료 방법을 찾기 힘들 수 있습니다. 두통은 뇌의 신경계가 예민해지면서 발생하는 질환입니다. 두통은 머리가 아픈 증상만 보이지 않으면서도, 증상이 심해질 경우에는 치료 방법을 찾기 힘들 수 있습니다. 두통은 뇌의 신경계가 예민해지면서 발생하는 질환입니다. 두통은 머리가 아픈 증상만 보이지 않으면서도, 증상이 심해질 경우에는 치료 방법을 찾기 힘들 수 있습니다.</s>기계가 멈추지 않아 계속 돌고 있습니다.
 한편, 김현수 통상교섭본부장은 한-인도 포괄적경제동반자협정(CEPA) 개선협


In [ ]:
import os
import torch
from transformers import (
    AutoModelForCausalLM, AutoTokenizer, # 사전 학습된 모델과 토크나이저를 불러오기 위한 클래스
    TrainingArguments, # 트레이닝 과정의 다양한 파라미터 세팅
    BitsAndBytesConfig, # 모델의 메모리 최적화 관련 설정
    pipeline,
    logging
)
from peft import LoraConfig
from trl import SFTTrainer

In [ ]:
query = '두통의 증상이 심해질 경우 어떤 치료 방법이 효과적일까요?'
pipe = pipeline(task='text-generation',
                model=model_load,
                tokenizer=tokenizer_load,
                max_length = 256)
result = pipe(f"<s>[INST] {query} [/INST]")
answer_result = result[0]['generated_text']

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [ ]:
answer_result

'<s>[INST] 두통의 증상이 심해질 경우 어떤 치료 방법이 효과적일까요? [/INST]두통은 다양한 이유로 발생할 수 있습니다. 두통은 일상적인 활동이나 스트레스와 관련된 요인으로 인해 증상이 나타나는 경향이 있습니다. 예를 들어, 두통은 피로, 우울감, 불안과 같은 감정 변화와 연관될 수 있습니다. 두통은 매우 흔한 증상이며, 다양한 증상과 함께 나타날 수 있습니다. 만약 두통이 지속된다면 의료 전문가와 상담하여 원인을 파악하고 적절한 치료 방법을 찾는 것이 중요합니다.</two>두통은 다양한 이유로 발생할 수 있습니다. 일상적인 활동이나 스트레스와 관련된 요인으로 인해 두통이 발생하는 경우가 많습니다. 만약 두통이 지속된다면 의료 전문가와 상담하여 원인을 파악하고 적절한 치료 방법을 찾아야 합니다.</two>'

In [ ]:
query = '머리 전체가 아픈 원인이 뭐야?'
pipe = pipeline(task='text-generation',
                model=model_load,
                tokenizer=tokenizer_load,
                max_length = 256)
result = pipe(f"<s>[INST] {query} [/INST]")
answer_result = result[0]['generated_text']
answer_result

'<s>[INST] 머리 전체가 아픈 원인이 뭐야? [/INST]두통은 머리나 목이 아픈 증상을 가리키는 용어입니다. 두통은 일반적으로 긴장성 두통과 편두통으로 나뉩니다. 긴장성 두통은 스트레스, 피로, 과로, 수면 부족 등으로 인해 발생하며, 주로 스트레스나 신경계 이상에 의해 발생합니다. 편두통은 신경성 통증으로 분류되며, 스트레스, 수면 부족, 약물 남용, 알코올 또는 커피와 같은 습관적인 약물, 감염, 특정 약물 등이 원인일 수 있습니다. 두통의 원인은 다양하며, 정확한 원인을 파악하여 적절한 치료와 예방이 필요합니다. 만약 두통이 계속되거나 심해진다면 의사와 상담하여 전문적인 치료를 받는 것이 좋습니다.</p>'